# 1. Starten met SQL

:::{admonition} Leerdoelen
:class: tip

In deze les leer je gegevens opvragen uit een relationele databank met SQL (Structured Query Language). Je werkt lokaal met SQLite en de tool DB Browser for SQLite.

Na deze les kan je:
- de basisopbouw van een query uitleggen: SELECT … FROM …
- resultaten filteren met WHERE
- resultaten sorteren met ORDER BY
- het aantal rijen beperken met LIMIT
- duplicaten verwijderen met DISTINCT
- berekende kolommen maken (bv. quantity * unit_price AS line_total)
- omgaan met tekst, datums (als tekst), NULL-waarden en patroonzoeken (LIKE)
- eenvoudige operatoren gebruiken: =, <, >, AND, OR, NOT, IN, BETWEEN 
:::


## 1. DB Browser: snel opstarten

1. Download en installeer [DB Browser for SQLite](https://sqlitebrowser.org/dl/).
2. Download de [database](/databases/webshop.db) voor deze les.
3. Start DB Browser for SQLite.
4. Open de database webshop.sqlite (Bestand → Open Database…).
5. Ga naar het tabblad Execute SQL: daar kan je SQL typen en uitvoeren met > Run. *Op het eind van deze les weet je hoe dat moet.*
6. Resultaten zie je onderaan in een tabel.

:::{tip}
**Krijg je een lege database, zonder tabellen?** Dan klikte je op de linkerknop *New Database*. Je hebt nu de database die je gedownload hebt overscheven met een lege database. Verwijder dat bestand en download het opnieuw.
:::

Commentaar in SQL (om je werk te documenteren):
 


In [ ]:
-- dit is commentaar

/* 
  dit is 
  meerregelige commentaar
*/



2. Tabellen in de webshop (overzicht)
    • customers(customer_id, first_name, last_name, email, city, postal_code, join_date)
    • products(product_id, name, category, unit_price, stock)
    • orders(order_id, customer_id, order_date, status)
    • order_lines(order_line_id, order_id, product_id, quantity, unit_price)
    
    Tip: Een rij = record, een kolom = veld, een tabel = verzameling rijen. Een PRIMARY KEY (PK) identificeert uniek een rij. Een FOREIGN KEY (FK) verwijst naar de PK van een andere tabel (daar komen we later op terug).

3. Je eerste query: SELECT & FROM
De eenvoudigste query vraagt kolommen op uit één tabel.

Code
-- Alle kolommen uit products (niet aan te raden voor echt werk)
SELECT * 
FROM products;
-- Enkele kolommen (beter)
SELECT name, unit_price 
FROM products;


Regels

• Elke query eindigt met een puntkomma ;
• Tabel- en kolomnamen precies spellen.
• Tekstwaarden tussen enkele aanhalingstekens: 'Antwerpen'.

4. Filteren met WHERE

Met WHERE kies je welke rijen je wil. 

Operatoren: =, <> (niet gelijk), <, >, <=, >=, AND, OR, NOT, ().

Code

-- Producten duurder dan 100 euro
SELECT name, unit_price
FROM products
WHERE unit_price > 100;

-- Klanten uit Antwerpen
SELECT first_name, last_name, city
FROM customers
WHERE city = 'Antwerpen';

-- Goedkope muizen of keyboards
SELECT name, unit_price
FROM products
WHERE (name LIKE '%Mouse%' OR name LIKE '%Keyboard%')
  AND unit_price < 50;

Datums in SQLite: vaak als tekst in ISO-vorm YYYY-MM-DD. Je kan dan filteren met LIKE '2025-07-%' voor alle dagen in juli 2025.

5. Sorteren met ORDER BY
Sorteer op één of meerdere kolommen. Standaard is ASC (oplopend); DESC is aflopend.

Code
-- Duurste producten eerst
SELECT name, unit_price
FROM products
ORDER BY unit_price DESC;

-- Klanten alfabetisch op achternaam, dan voornaam
SELECT first_name, last_name
FROM customers
ORDER BY last_name ASC, first_name ASC;

6. Aantal rijen beperken met LIMIT
Code
-- Alleen de 5 duurste producten
SELECT name, unit_price
FROM products
ORDER BY unit_price DESC
LIMIT 5;

    Extra: OFFSET n overslaat de eerste n rijen (paginatie). Niet nodig vandaag.

7. Duplicaten verwijderen met DISTINCT
Code
-- Unieke lijst van steden waar klanten wonen
SELECT DISTINCT city
FROM customers
ORDER BY city;

8. Berekende kolommen + aliassen (AS)
Je kan rekenen in de SELECT en de kolom een bijnaam geven met AS.

Code
-- Totaalbedrag per orderregel (nog géén optelling per order!)
SELECT order_line_id,
       quantity * unit_price AS line_total
FROM order_lines;

-- Eenvoudige tekstbewerking: alles in kleine letters
SELECT LOWER(name) AS productnaam_klein
FROM products;

    Taalpuntje: Gebruik punt als decimaalteken in SQL (19.9), geen komma.



9. Speciale waarden: NULL (onbekend/ontbreekt)
NULL is geen lege tekst en ook niet 0. Je vergelijkt het niet met = maar met IS NULL of IS NOT NULL.

Code
-- Klanten zonder e-mailadres
SELECT first_name, last_name, email
FROM customers
WHERE email IS NULL;

-- Klanten met e-mailadres
SELECT first_name, last_name, email
FROM customers
WHERE email IS NOT NULL;


10. Patronen zoeken met LIKE
LIKE gebruikt jokertekens:
    • % = willekeurige tekenreeks (ook leeg)
    • _ = precies één willekeurig teken
    
Code
-- Productnamen die 'Pro' bevatten
SELECT name
FROM products
WHERE name LIKE '%Pro%';

-- Postcodes die starten met '20'
SELECT first_name, last_name, postal_code
FROM customers
WHERE postal_code LIKE '20__';  -- '20' + exact twee tekens

    Hoofdletters/kleine letters: LIKE kan per databank anders omgaan met case. Wil je zeker case-insensitief zoeken, zet beide kanten naar kleine letters:

Code
WHERE LOWER(name) LIKE LOWER('%Pro%');

11. IN, NOT IN en BETWEEN
Handig om te filteren op lijsten of bereiken.

Code
-- Producten in bepaalde categorieën
SELECT name, category, unit_price
FROM products
WHERE category IN ('Gaming', 'Audio', 'Displays');

-- Niet in die categorieën
WHERE category NOT IN ('Gaming', 'Audio', 'Displays');

-- Unit price tussen 100 en 300 (grenzen inbegrepen)
WHERE unit_price BETWEEN 100 AND 300;


12. Datums filteren (als tekst)
We gebruiken ISO-datums (YYYY-MM-DD) in orders.order_date.

Code
-- Alle bestellingen in juli 2025
SELECT order_id, order_date, status
FROM orders
WHERE order_date LIKE '2025-07-%'
ORDER BY order_date;

-- Bestellingen vanaf 1 april 2025 (tekstvergelijking werkt dankzij ISO-vorm)
WHERE order_date >= '2025-04-01';

13. Veelgemaakte fouten (checklist)

    Puntkomma vergeten ; → query voert niet uit.
    Tekst zonder aanhalingstekens: city = Antwerpen → moet zijn city = 'Antwerpen'.
    Komma’s i.p.v. punten in getallen: 19,9 → moet 19.9 zijn.
    NULL vergelijken met = → gebruik IS NULL of IS NOT NULL.
    Kolomnaam verkeerd gespeld → controleer schema.
    ORDER BY voor WHERE zetten → volgorde is: SELECT … FROM … WHERE … ORDER BY … LIMIT ….
    

